In [ ]:
pip install -U langchain-community

In [ ]:
import os
from getpass import getpass

os.environ["HF_TOKEN"] = getpass("Enter your Hugging Face Token: ")

Enter your Hugging Face Token: ··········


In [ ]:
!pip install langchain_groq # to access LLMs

In [ ]:
pip install pypdf

In [ ]:
!pip install gradio

In [ ]:
!pip install chromadb

In [ ]:
groq_api_key = getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
)

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings # create vector embeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader # extract text from pdf files
from langchain.vectorstores import Chroma # stores vector embeddings in a vector db
from langchain.chains import RetrievalQA # Combines a retriever (like Chroma) with a language model to generate answers based on retrieved documents.
from langchain.prompts import PromptTemplate # Structures input to ensure the model generates the desired output.
from langchain.text_splitter import RecursiveCharacterTextSplitter # Splits large text documents into smaller chunks for more efficiency
import gradio as gr

In [ ]:
def initialize_llm():
  llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
)
  return llm

def create_vector_db():
  loader = DirectoryLoader('/content/data/', glob='*.pdf', loader_cls=PyPDFLoader) # can contain multiple documents
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
  vector_db.persist()

  print("Chroma DB created and data saved")

  return vector_db
  # persist_directory specifies the directory where the vector database will be stored on disk. This allows you to save and reload the vector embeddings instead of recomputing them every time.


def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """

  PROMPT = PromptTemplate(
      template=prompt_templates, input_variables=["context", "question"]
  )

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={"prompt": PROMPT},
  )

  return qa_chain


def main():
  print("Initializing LLM...")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    print("Creating vector database...")
    vector_db = create_vector_db()

  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    # ChromaDB already contains the stored embeddings and document mappings. It retrieves the data directly from disk instead of processing PDFs again.

  print("Setting up QA chain...")
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:

    query = input("\nHuman: ")
    if query.lower() == "exit":
      print("Exiting the chatbot.")
      break

    result = qa_chain({"query": query})
    answer = result["result"]
    print(f"\nChatbot: {answer}")


if __name__ == "__main__":
  main()


Initializing LLM...
Setting up QA chain...

Human: hi

Chatbot: Hello. It seems like you were reading about the definition of mental health, specifically the concept of a "good life" and how it's perceived by individuals. The text you provided mentions that people have their own unique interpretations of what a fulfilling life is, and that mental health is often defined as having a sense of purpose, quality relationships, self-respect, and mastery.

I'd like to explore this topic further with you. What are your thoughts on what constitutes a "good life"? How do you think an individual's social and physical environment influences their mental health and well-being? Is there something specific that's been on your mind lately, or would you like to discuss this topic in general? I'm here to listen and offer support.

Human: i am depressed 

Chatbot: I'm so sorry to hear that you're feeling depressed. It takes a lot of courage to acknowledge and share your emotions, and I'm here to listen a

In [ ]:
def initialize_llm():
  llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile",
)
  return llm

def create_vector_db():
  loader = DirectoryLoader('/content/data/', glob='*.pdf', loader_cls=PyPDFLoader) # can contain multiple documents
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory="/content/chroma_db")
  vector_db.persist()

  print("Chroma DB created and data saved")

  return vector_db
  # persist_directory specifies the directory where the vector database will be stored on disk. This allows you to save and reload the vector embeddings instead of recomputing them every time.


def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """

  PROMPT = PromptTemplate(
      template=prompt_templates, input_variables=["context", "question"]
  )

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={"prompt": PROMPT},
  )

  return qa_chain

print("Intializing Chatbot.........")
llm = initialize_llm()

db_path = "/content/chroma_db"

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history = []):
  if not user_input.strip():
    return "Please provide a valid input", history
  result = qa_chain({"query": user_input})
  answer = result["result"]
  history.append((user_input, answer))
  return ""

with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
    gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
    gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.")

    chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

    gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

app.launch()

Intializing Chatbot.........


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0afc61f1f5143b08ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
